In [0]:
from databricks.feature_store import FeatureStoreClient

In [0]:
# Nuevas columnas de la tabla base_consolidada
feature_columns = [
    'periodo', 'id_cliente', 'tiempo_permanencia', 'flg_vip', 'incidencias_a', 'incidencias_b', 
    'tipo_producto', 'periodo_creacion', 'departamento', 'segmento_pago', 'canal', 
    'segmento_cliente', 'crossell', 'tasa', 'monto_1m', 'monto_2m', 'monto_3m', 'monto_4m', 
    'monto_5m', 'monto_6m', 'cantidad_1m', 'cantidad_2m', 'cantidad_3m', 'cantidad_6m', 
    'frecuencia_1m', 'frecuencia_2m', 'frecuencia_3m', 'ultima_compra_1m', 'ultima_compra_2m', 
    'ultima_compra_3m', 'monto_total', 'tendencia_monto'
]

# Variable objetivo (label)
flag = "flg_churn"

print("🚀 Columnas seleccionadas para entrenamiento:", feature_columns)
print("🎯 Variable objetivo:", flag)


🚀 Columnas seleccionadas para entrenamiento: ['periodo', 'id_cliente', 'tiempo_permanencia', 'flg_vip', 'incidencias_a', 'incidencias_b', 'tipo_producto', 'periodo_creacion', 'departamento', 'segmento_pago', 'canal', 'segmento_cliente', 'crossell', 'tasa', 'monto_1m', 'monto_2m', 'monto_3m', 'monto_4m', 'monto_5m', 'monto_6m', 'cantidad_1m', 'cantidad_2m', 'cantidad_3m', 'cantidad_6m', 'frecuencia_1m', 'frecuencia_2m', 'frecuencia_3m', 'ultima_compra_1m', 'ultima_compra_2m', 'ultima_compra_3m', 'monto_total', 'tendencia_monto']
🎯 Variable objetivo: flg_churn


In [0]:
fs = FeatureStoreClient()
feature_table_name = "databricks_clase.prueba_schema.base_consolidada_mensual_feats"
df_base = spark.read.table("databricks_clase.prueba_schema.base_consolidada_mensual")

In [0]:

# creamos la tabla en el Feature Store
fs.create_table(
    name=feature_table_name,
    primary_keys=["id_cliente"],  # Clave primaria
    schema=df_base.select(*feature_columns).schema,  # Solo con las columnas necesarias
    description="Feature store con todas las columnas del dataset consolidado (excluyendo flg_churn)"
)


2025/02/26 00:57:08 INFO databricks.ml_features._compute_client._compute_client: Setting columns ['id_cliente'] of table 'databricks_clase.prueba_schema.base_consolidada_mensual_feats' to NOT NULL.
2025/02/26 00:57:10 INFO databricks.ml_features._compute_client._compute_client: Setting Primary Keys constraint ['id_cliente'] on table 'databricks_clase.prueba_schema.base_consolidada_mensual_feats'.
/databricks/python/lib/python3.12/site-packages/databricks/sdk/service/jobs.py:60: SyntaxWarning: invalid escape sequence '\.'
  """The sequence number of this run attempt for a triggered job run. The initial attempt of a run
/databricks/python/lib/python3.12/site-packages/databricks/sdk/service/jobs.py:2570: SyntaxWarning: invalid escape sequence '\.'
  """The sequence number of this run attempt for a triggered job run. The initial attempt of a run
/databricks/python/lib/python3.12/site-packages/databricks/sdk/service/jobs.py:3431: SyntaxWarning: invalid escape sequence '\.'
  """The sequence

<FeatureTable: name='databricks_clase.prueba_schema.base_consolidada_mensual_feats', table_id='e59d428c-c555-454f-8e13-21c677de4e65', description=('Feature store con todas las columnas del dataset consolidado (excluyendo '
 'flg_churn)'), primary_keys=['id_cliente'], partition_columns=[], features=['periodo',
 'id_cliente',
 'tiempo_permanencia',
 'flg_vip',
 'incidencias_a',
 'incidencias_b',
 'tipo_producto',
 'periodo_creacion',
 'departamento',
 'segmento_pago',
 'canal',
 'segmento_cliente',
 'crossell',
 'tasa',
 'monto_1m',
 'monto_2m',
 'monto_3m',
 'monto_4m',
 'monto_5m',
 'monto_6m',
 'cantidad_1m',
 'cantidad_2m',
 'cantidad_3m',
 'cantidad_6m',
 'frecuencia_1m',
 'frecuencia_2m',
 'frecuencia_3m',
 'ultima_compra_1m',
 'ultima_compra_2m',
 'ultima_compra_3m',
 'monto_total',
 'tendencia_monto'], creation_timestamp=1740531428445, online_stores=[], notebook_producers=[], job_producers=[], table_data_sources=[], path_data_sources=[], custom_data_sources=[], timestamp_keys=[],

In [0]:
# Excluir 'id_cliente' de las columnas si ya es clave primaria
columns_to_write = [col for col in feature_columns if col != "id_cliente"]

# Escribir la tabla en el Feature Store
fs.write_table(
    name=feature_table_name,
    df=df_base.select("id_cliente", *columns_to_write),  # 'id_cliente' ya está como clave primaria
    mode="overwrite"
)

In [0]:
# leer tabla de feature store
features_df = fs.read_table(name=feature_table_name)

# Convertir a Pandas para usar con sklearn
features = features_df.toPandas()

In [0]:
import pandas as pd

#  Recuperar el target desde el dataset base
target = df_base.select("id_cliente", "flg_churn").toPandas()

#  Asegurar que no haya duplicados en `id_cliente`
features = features.drop_duplicates(subset=['id_cliente'])
target = target.drop_duplicates(subset=['id_cliente'])

#  Verificar tamaños antes del merge
print(f"Antes del merge - features: {features.shape}, target: {target.shape}")

#  Hacer la unión con verificación de nulos
features_with_target = features.merge(target, on="id_cliente", how="inner")
print(f"Después del merge: {features_with_target.shape}")
print("Valores nulos después del merge:\n", features_with_target.isnull().sum())



Antes del merge - features: (48161, 32), target: (48161, 2)
Después del merge: (48161, 33)
Valores nulos después del merge:
 periodo               0
id_cliente            0
tiempo_permanencia    0
flg_vip               0
incidencias_a         0
incidencias_b         0
tipo_producto         0
periodo_creacion      0
departamento          0
segmento_pago         0
canal                 0
segmento_cliente      0
crossell              0
tasa                  0
monto_1m              0
monto_2m              0
monto_3m              0
monto_4m              0
monto_5m              0
monto_6m              0
cantidad_1m           0
cantidad_2m           0
cantidad_3m           0
cantidad_6m           0
frecuencia_1m         0
frecuencia_2m         0
frecuencia_3m         0
ultima_compra_1m      0
ultima_compra_2m      0
ultima_compra_3m      0
monto_total           0
tendencia_monto       0
flg_churn             0
dtype: int64


In [0]:
#  Separar features y target
X = features_with_target[['frecuencia_1m', 'cantidad_1m', 'tasa', 'ultima_compra_1m', 'monto_total']]
y = features_with_target["flg_churn"]

#  Verificar dimensiones finales
print(f"Dimensiones finales - X: {X.shape}, y: {y.shape}")
print(f"Distribución de y:\n{y.value_counts(normalize=True)}")


Dimensiones finales - X: (48161, 5), y: (48161,)
Distribución de y:
0.0    0.873279
1.0    0.126721
Name: flg_churn, dtype: float64


In [0]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import mlflow

In [0]:
# 1 Convertir variables categóricas en numéricas
categorical_cols = ['tipo_producto', 'departamento', 'canal']
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    features_with_target[col] = le.fit_transform(features_with_target[col])
    label_encoders[col] = le  # Guardar el encoder para futuras conversiones

# 2 Manejo de valores nulos: Llenar con la media
features_with_target.fillna(features_with_target.mean(), inplace=True)

# 3 Escalar las características (opcional pero recomendado)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4 Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# 5 Crear y entrenar el modelo
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight="balanced")
model.fit(X_train, y_train)

# 6 Predicción
y_pred = model.predict(X_test)

# 7 Evaluación
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print(f" F1 Score: {f1:.4f}")
print(f" Accuracy: {accuracy:.4f}")
print(" Classification Report:\n", classification_report(y_test, y_pred))

Uploading artifacts:   0%|          | 0/3 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

 F1 Score: 0.5178
 Accuracy: 0.8086
 Classification Report:
               precision    recall  f1-score   support

         0.0       0.97      0.81      0.88      8412
         1.0       0.38      0.81      0.52      1221

    accuracy                           0.81      9633
   macro avg       0.67      0.81      0.70      9633
weighted avg       0.89      0.81      0.83      9633



In [0]:
mlflow.end_run()

In [0]:
mlflow.sklearn.autolog(disable=True)

with mlflow.start_run():
    # 1 Convertir variables categóricas en numéricas
    categorical_cols = ['tipo_producto', 'departamento', 'canal']
    label_encoders = {}

    for col in categorical_cols:
        le = LabelEncoder()
        features_with_target[col] = le.fit_transform(features_with_target[col])
        label_encoders[col] = le  # Guardar el encoder para futuras conversiones

    # 2 Manejo de valores nulos: Llenar con la media
    features_with_target.fillna(features_with_target.mean(), inplace=True)

    # 3 Escalar las características (opcional pero recomendado)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 4 Dividir en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

    # 5 Crear y entrenar el modelo
    model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight="balanced")
    model.fit(X_train, y_train)
    mlflow.log_param("parametro_adicional", "valor_ejemplo")

    # 6 Predicción
    y_pred = model.predict(X_test)
    mlflow.sklearn.log_model(model, "model")
    # 7 Evaluación
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_param("f1", f1)
    mlflow.log_param("accuracy", accuracy)
    print(f" F1 Score: {f1:.4f}")
    print(f" Accuracy: {accuracy:.4f}")
    print(" Classification Report:\n", classification_report(y_test, y_pred))
mlflow.end_run()


2025/02/26 01:11:23 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.15.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

2025/02/26 01:11:24 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025/02/26 01:11:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run debonair-midge-217 at: adb-106485471189205.5.azuredatabricks.net/ml/experiments/95e525fba0004e0ab39e31ec27252768/runs/c01904e839e145aca1bd036fe95e62f9.
2025/02/26 01:11:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: adb-106485471189205.5.azuredatabricks.net/ml/experiments/95e525fba0004e0ab39e31ec27252768.


 F1 Score: 0.5178
 Accuracy: 0.8086
 Classification Report:
               precision    recall  f1-score   support

         0.0       0.97      0.81      0.88      8412
         1.0       0.38      0.81      0.52      1221

    accuracy                           0.81      9633
   macro avg       0.67      0.81      0.70      9633
weighted avg       0.89      0.81      0.83      9633



In [0]:
X

,frecuencia_1m,cantidad_1m,tasa,ultima_compra_1m,monto_total
0,28.0,4.499810e+00,0.034400,30.0,49.033113
1,1.0,1.000000e-07,0.028000,1.0,32.406695
2,5.0,1.791759e+00,0.034400,30.0,43.357694
3,3.0,2.397895e+00,0.034400,22.0,24.500427
4,25.0,4.615121e+00,0.034400,30.0,49.986442
...,...,...,...,...,...
48156,2.0,6.931472e-01,0.031509,25.0,-78.922771
48157,3.0,1.098612e+00,0.034400,22.0,-57.107202
48158,5.0,1.609438e+00,0.029500,30.0,-52.690897
48159,4.0,1.609438e+00,0.034400,22.0,38.718901


In [0]:
model_uri = 'runs:/c01904e839e145aca1bd036fe95e62f9/model'

#  Definir un input correcto con las columnas exactas que espera el modelo
input_example = pd.DataFrame([{
    "frecuencia_1m": 4,
    "cantidad_1m": 8,
    "tasa": 0.33,
    "ultima_compra_1m": 15,
    "monto_total": 5000,
}])


#  Convertir todas las columnas a float64
input_example = input_example.astype('float64')

#  Cargar el modelo y predecir
predicciones = mlflow.pyfunc.load_model(model_uri).predict(input_example)

#  Mostrar resultado
print(" Predicción:", predicciones)


 Predicción: [0.]


/databricks/python/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
